In [11]:
import sys
import functools

def trace(func=None, *, handle=sys.stdout):
    print(f"decorated func: {func}, {handle}")
    if func is None:
        print('func is None')
        return lambda func: trace(func, handle=handle)
    else:
        print(f'{func.__name__}, {handle}')

    @functools.wraps(func)
    def inner(*args, **kwargs):
        handle.write(f"Using handling output\n")
        # print(func.__name__, args, kwargs)
        return func(*args, **kwargs)

    # print('return inner')
    return inner

In [12]:
import io
import functools


nonstandardstream = io.StringIO()
@trace(handle=nonstandardstream)
def increm(x):
    """Инкремент"""
    # print("Инкремент")
    return x+1

increm(2)

nonstandardstream.getvalue()

decorated func: None, <_io.StringIO object at 0x000002293495B700>
func is None
decorated func: <function increm at 0x00000229349AED40>, <_io.StringIO object at 0x000002293495B700>
increm, <_io.StringIO object at 0x000002293495B700>


'Using handling output\n'

In [13]:
import functools
import requests
import sys
import io

def get_currencies(currency_codes: list, url:str = "https://www.cbr-xml-daily.ru/daily_json.js", handle=sys.stdout)->dict:
    """
    Получает курсы валют с API Центробанка России.

    Args:
        currency_codes (list): Список символьных кодов валют (например, ['USD', 'EUR']).

    Returns:
        dict: Словарь, где ключи - символьные коды валют, а значения - их курсы.
              Возвращает None в случае ошибки запроса.
    """
    try:

        response = requests.get(url)

        # print(response.status_code)
        response.raise_for_status()  # Проверка на ошибки HTTP
        data = response.json()
        # print(data)
        currencies = {}

        if "Valute" in data:
            for code in currency_codes:
                if code in data["Valute"]:
                    currencies[code] = data["Valute"][code]["Value"]
                else:
                    currencies[code] = f"Код валюты '{code}' не найден."
        return currencies

    except requests.exceptions.RequestException as e:
        # print(f"Ошибка при запросе к API: {e}", file=handle)
        handle.write(f"Ошибка при запросе к API: {e}")
        # raise ValueError('Упали с исключением')
        raise requests.exceptions.RequestException('Упали с исключением')

# Пример использования функции:
currency_list = ['USD', 'EUR', 'GBP', 'NNZ']

currency_data = get_currencies(currency_list)

if currency_data:
     print(currency_data)

{'USD': 80.601, 'EUR': 93.6953, 'GBP': 105.8291, 'NNZ': "Код валюты 'NNZ' не найден."}


In [14]:
import unittest
from requests import *
import io

MAX_R_VALUE = 1000


# Тесты
class TestGetCurrencies(unittest.TestCase):

  def test_currency_usd(self):
    """
      Проверяет наличие ключа в словаре и значения этого ключа
    """
    currency_list = ['USD']
    currency_data = get_currencies(currency_list)

    self.assertIn(currency_list[0], currency_data)
    self.assertIsInstance(currency_data['USD'], float)
    self.assertGreaterEqual(currency_data['USD'], 0)
    self.assertLessEqual(currency_data['USD'], MAX_R_VALUE)

  def test_nonexist_code(self):
    self.assertIn("Код валюты", get_currencies(['XYZ'])['XYZ'])
    self.assertIn("XYZ", get_currencies(['XYZ'])['XYZ'])
    self.assertIn("не найден", get_currencies(['XYZ'])['XYZ'])

  def test_get_currency_error(self):
    error_phrase_regex = "Ошибка при запросе к API"
    currency_io = io.StringIO()

    with self.assertRaises(requests.exceptions.RequestException):
      get_currencies(currency_list,url="https://",handle=currency_io)
    self.assertIn(error_phrase_regex, currency_io.getvalue())



  #   # Найти каким образом проверить содержание фразы error_phase_regex в
  #   # потоке вывода

  #   # Дополнить тест, который должен проверять что в потоке, куда пишет функция
  #   # get_currencies содержится error_phrase_regex /
  #   # для использования assertStartsWith или assertRegex





# Запуск тестов
unittest.main(argv=[''], verbosity=2, exit=False)

test_currency_usd (__main__.TestGetCurrencies.test_currency_usd)
Проверяет наличие ключа в словаре и значения этого ключа ... ok
test_get_currency_error (__main__.TestGetCurrencies.test_get_currency_error) ... ok
test_nonexist_code (__main__.TestGetCurrencies.test_nonexist_code) ... ok
test_writing_stream (__main__.TestStreamWrite.test_writing_stream) ... ok

----------------------------------------------------------------------
Ran 4 tests in 1.373s

OK


In [15]:
import sys

def trace(func=None, *, handle=sys.stdout):
    # print(f"decorated func: {func}, {handle}")
    if func is None:
        # print('func is None')
        return lambda func: trace(func, handle=handle)
    # else:
    #   print(f'{func.__name__}, {handle}')

    @functools.wraps(func)
    def inner(*args, **kwargs):
        handle.write(f"Using handling output\n")
        # print(func.__name__, args, kwargs)
        return func(*args, **kwargs)

    # print('return inner')
    return inner

In [16]:
import sys

def trace(func=None, *, handle=sys.stdout):
    # print(f"decorated func: {func}, {handle}")
    if func is None:
        # print('func is None')
        return lambda func: trace(func, handle=handle)
    # else:
    #   print(f'{func.__name__}, {handle}')

    @functools.wraps(func)
    def inner(*args, **kwargs):
        handle.write(f"Using handling output\n")
        # print(func.__name__, args, kwargs)
        return func(*args, **kwargs)

    # print('return inner')
    return inner

In [17]:
import unittest
import io


# Тесты
class TestStreamWrite(unittest.TestCase):


  def setUp(self):
    self.nonstandardstream = io.StringIO()


    try:
      self.get_currencies = get_currencies(['USD'],
                                         url="https://",
                                         handle=self.nonstandardstream)
    except:
      pass
    # self.trace = trace(get_currencies, handle=self.nonstandardstream)


  def test_writing_stream(self):
    pass
    #with self.assertRaises(requests.exceptions.RequestException):
     # self.trace(['USD'], url="https://")


  def tearDown(self):
    del self.nonstandardstream




# Запуск тестов
unittest.main(argv=[''], verbosity=2, exit=False)

test_currency_usd (__main__.TestGetCurrencies.test_currency_usd)
Проверяет наличие ключа в словаре и значения этого ключа ... ok
test_get_currency_error (__main__.TestGetCurrencies.test_get_currency_error) ... ok
test_nonexist_code (__main__.TestGetCurrencies.test_nonexist_code) ... ok
test_writing_stream (__main__.TestStreamWrite.test_writing_stream) ... ok

----------------------------------------------------------------------
Ran 4 tests in 1.294s

OK


In [18]:
currency_data = get_currencies(currency_list, url="sdfhsjdsdfsdfm")

Ошибка при запросе к API: Invalid URL 'sdfhsjdsdfsdfm': No scheme supplied. Perhaps you meant https://sdfhsjdsdfsdfm?

RequestException: Упали с исключением

In [9]:
import unittest
import io



def super_sum(a, b, file='my_file.txt'):
  f = open(file, mode='w', encoding='utf-8')
  res = a + b
  f.write(f'super_sum, {a}, {b}, result: {res}')
  f.close()

  return res



super_sum(100, 200 )
# Тесты
class TestStreamWrite(unittest.TestCase):

  def setUp(self):
    self.file_for_tests = 'file_for_tests.txt'

  def test_writing_stream(self):

    args = (100, 200)
    exp_result_tmpl = 'название_функции, 100, 200, result: 300'

    exp_result_func = super_sum(*args, file=self.file_for_tests)

    exp_result = f"{super_sum.__name__}, " + ', '.join(map (lambda x: str(x),list(args))) + f", result: {exp_result_func}"

    self.f_t = open(self.file_for_tests)
    self.exp_result = self.f_t.read().strip()
    self.assertEqual(exp_result, self.exp_result)

  def tearDown(self):
    self.f_t.close()
    del self.file_for_tests




# Запуск тестов
unittest.main(argv=[''], verbosity=2, exit=False)

test_currency_usd (__main__.TestGetCurrencies.test_currency_usd)
Проверяет наличие ключа в словаре и значения этого ключа ... ok
test_get_currency_error (__main__.TestGetCurrencies.test_get_currency_error) ... ok
test_nonexist_code (__main__.TestGetCurrencies.test_nonexist_code) ... ok
test_writing_stream (__main__.TestStreamWrite.test_writing_stream) ... ok

----------------------------------------------------------------------
Ran 4 tests in 1.873s

OK


In [10]:
def super_sum(a, b, file='my_file.txt'):
  f = open(file, mode='w', encoding='utf-8')
  res = a + b
  f.write(f'super_sum, {a}, {b}, result: {res}')
  f.close()

  return res



# print(super_sum('1', '2'))


try:
  super_sum('1' , 2)
except (ValueError, TypeError) as e:
  print(str(e))
  f = open('errors.log', mode='w')
  f.write(str(e))
  f.close()


# print("контекст ошибки: ", e)

can only concatenate str (not "int") to str
